# MakeItTalk Quick Demo (natural human face animation)

- included project setup + pretrained model download
- provides step-by-step details
- todo: tdlr version

## Preparations
- Check GPU

In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi

- Check ffmpeg

- Install Github https://github.com/yzhou359/MakeItTalk

In [2]:
!git clone https://github.com/yzhou359/MakeItTalk

Cloning into 'MakeItTalk'...
remote: Enumerating objects: 650, done.
remote: Total 650 (delta 0), reused 0 (delta 0), pack-reused 650
Receiving objects: 100% (650/650), 26.21 MiB | 36.42 MiB/s, done.
Resolving deltas: 100% (171/171), done.


- Install requirements

In [3]:
%cd MakeItTalk/
!export PYTHONPATH=/content/MakeItTalk:$PYTHONPATH
!pip install fastapi[all] nest-asyncio pyngrok uvicorn python-multipart==0.0.4
!pip install -r requirements.txt
!pip install tensorboardX

/content/MakeItTalk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 745 kB 15.9 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 272 kB 71.7 MB/s 
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |███████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 7.5 MB/s 


- Download pretrained models

In [4]:
!mkdir examples/dump
!mkdir examples/ckpt
!pip install gdown
!gdown -O examples/ckpt/ckpt_autovc.pth https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
!gdown -O examples/ckpt/ckpt_content_branch.pth https://drive.google.com/uc?id=1r3bfEvTVl6pCNw5xwUhEglwDHjWtAqQp
!gdown -O examples/ckpt/ckpt_speaker_branch.pth https://drive.google.com/uc?id=1rV0jkyDqPW-aDJcj7xSO6Zt1zSXqn1mu
!gdown -O examples/ckpt/ckpt_116_i2i_comb.pth https://drive.google.com/uc?id=1i2LJXKp-yWKIEEgJ7C6cE3_2NirfY_0a
!gdown -O examples/dump/emb.pickle https://drive.google.com/uc?id=18-0CYl5E6ungS3H4rRSHjfYvvm-WwjTI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
To: /content/MakeItTalk/examples/ckpt/ckpt_autovc.pth
100% 172M/172M [00:00<00:00, 230MB/s]
Downloading...
From: https://drive.google.com/uc?id=1r3bfEvTVl6pCNw5xwUhEglwDHjWtAqQp
To: /content/MakeItTalk/examples/ckpt/ckpt_content_branch.pth
100% 7.88M/7.88M [00:00<00:00, 124MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rV0jkyDqPW-aDJcj7xSO6Zt1zSXqn1mu
To: /content/MakeItTalk/examples/ckpt/ckpt_speaker_branch.pth
100% 15.4M/15.4M [00:00<00:00, 142MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i2LJXKp-yWKIEEgJ7C6cE3_2NirfY_0a
To: /content/MakeItTalk/examples/ckpt/ckpt_116_i2i_comb.pth
100% 839M/839M [00:06<00:00, 122MB/s]
Downloading...
From: https://drive.google.com/uc?id=18-0CYl5E6ungS3H4rRSHjfYvvm-WwjTI
To: /content/MakeItTalk/examples/dump/emb.pickle
100% 30.9M/30.9M [00:00<00:0

In [ ]:
!ngrok authtoken YOUR_TOKEN          #Please get your token by singing up at ngrok.com and replace by YOUR_TOKEN

- prepare your images/audios (or you can use the existing ones)
  - An image to animate: upload to `MakeItTalk/examples` folder, image size should be 256x256
  - An audio (hopefully no noise) to talk: upload to `MakeItTalk/examples` folder as well

## Step 0: import necessary packages

In [5]:
import sys
sys.path.append("thirdparty/AdaptiveWingLoss")
import os, glob
import numpy as np
import cv2
import argparse
from src.approaches.train_image_translation import Image_translation_block
import torch
import pickle
import face_alignment
from src.autovc.AutoVC_mel_Convertor_retrain_version import AutoVC_mel_Convertor
import shutil
import time
import util.utils as util
from scipy.signal import savgol_filter
from src.approaches.train_audio2landmark import Audio2landmark_model


ADD_NAIVE_EYE = True                 # whether add naive eye blink
CLOSE_INPUT_FACE_MOUTH = False       # if your image has an opened mouth, put this as True, else False
AMP_LIP_SHAPE_X = 2.                 # amplify the lip motion in horizontal direction
AMP_LIP_SHAPE_Y = 2.                 # amplify the lip motion in vertical direction
AMP_HEAD_POSE_MOTION = 0.7           # amplify the head pose motion (usually smaller than 1.0, put it to 0. for a static head pose)

parser = argparse.ArgumentParser()
parser.add_argument('--close_input_face_mouth', default=CLOSE_INPUT_FACE_MOUTH, action='store_true')

parser.add_argument('--load_AUTOVC_name', type=str, default='examples/ckpt/ckpt_autovc.pth')
parser.add_argument('--load_a2l_G_name', type=str, default='examples/ckpt/ckpt_speaker_branch.pth')
parser.add_argument('--load_a2l_C_name', type=str, default='examples/ckpt/ckpt_content_branch.pth') #ckpt_audio2landmark_c.pth')
parser.add_argument('--load_G_name', type=str, default='examples/ckpt/ckpt_116_i2i_comb.pth') #ckpt_image2image.pth') #ckpt_i2i_finetune_150.pth') #c

parser.add_argument('--amp_lip_x', type=float, default=AMP_LIP_SHAPE_X)
parser.add_argument('--amp_lip_y', type=float, default=AMP_LIP_SHAPE_Y)
parser.add_argument('--amp_pos', type=float, default=AMP_HEAD_POSE_MOTION)
parser.add_argument('--reuse_train_emb_list', type=str, nargs='+', default=[]) #  ['iWeklsXc0H8']) #['45hn7-LXDX8']) #['E_kmpT-EfOg']) #'iWeklsXc0H8', '29k8RtSUjE0', '45hn7-LXDX8',
parser.add_argument('--add_audio_in', default=False, action='store_true')
parser.add_argument('--comb_fan_awing', default=False, action='store_true')
parser.add_argument('--output_folder', type=str, default='examples')

parser.add_argument('--test_end2end', default=True, action='store_true')
parser.add_argument('--dump_dir', type=str, default='', help='')
parser.add_argument('--pos_dim', default=7, type=int)
parser.add_argument('--use_prior_net', default=True, action='store_true')
parser.add_argument('--transformer_d_model', default=32, type=int)
parser.add_argument('--transformer_N', default=2, type=int)
parser.add_argument('--transformer_heads', default=2, type=int)
parser.add_argument('--spk_emb_enc_size', default=16, type=int)
parser.add_argument('--init_content_encoder', type=str, default='')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--reg_lr', type=float, default=1e-6, help='weight decay')
parser.add_argument('--write', default=False, action='store_true')
parser.add_argument('--segment_batch_size', type=int, default=1, help='batch size')
parser.add_argument('--emb_coef', default=3.0, type=float)
parser.add_argument('--lambda_laplacian_smooth_loss', default=1.0, type=float)
parser.add_argument('--use_11spk_only', default=False, action='store_true')
parser.add_argument('-f')

opt_parser = parser.parse_args()

def detect_landmarks(img):
  predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, device='cpu', flip_input=True)
  shapes = predictor.get_landmarks(img)
  if (not shapes or len(shapes) != 1):
      print('Cannot detect face landmarks. Exit.')
      exit(-1)
  shape_3d = shapes[0]

  if(opt_parser.close_input_face_mouth):
      util.close_input_face_mouth(shape_3d)
  shape_3d[48:, 0] = (shape_3d[48:, 0] - np.mean(shape_3d[48:, 0])) * 1.05 + np.mean(shape_3d[48:, 0]) # wider lips
  shape_3d[49:54, 1] += 0.           # thinner upper lip
  shape_3d[55:60, 1] -= 1.           # thinner lower lip
  shape_3d[[37,38,43,44], 1] -=2.    # larger eyes
  shape_3d[[40,41,46,47], 1] +=2.    # larger eyes
  shape_3d, scale, shift = util.norm_input_face(shape_3d)
  return shape_3d, scale, shift, img

def gen_audio():
  au_data = []
  au_emb = []
  ains = glob.glob1('examples', '*.wav')
  ains = [item for item in ains if item is not 'tmp.wav']
  ains.sort()
  for ain in ains:
      os.system('ffmpeg -y -loglevel error -i examples/{} -ar 16000 examples/tmp.wav'.format(ain))
      shutil.copyfile('examples/tmp.wav', 'examples/{}'.format(ain))

      # au embedding
      from thirdparty.resemblyer_util.speaker_emb import get_spk_emb
      me, ae = get_spk_emb('examples/{}'.format(ain))
      au_emb.append(me.reshape(-1))

      print('Processing audio file', ain)
      c = AutoVC_mel_Convertor('examples')

      au_data_i = c.convert_single_wav_to_autovc_input(audio_filename=os.path.join('examples', ain),
            autovc_model_path=opt_parser.load_AUTOVC_name)
      au_data += au_data_i
  if(os.path.isfile('examples/tmp.wav')):
      os.remove('examples/tmp.wav')

  # landmark fake placeholder
  fl_data = []
  rot_tran, rot_quat, anchor_t_shape = [], [], []
  for au, info in au_data:
      au_length = au.shape[0]
      fl = np.zeros(shape=(au_length, 68 * 3))
      fl_data.append((fl, info))
      rot_tran.append(np.zeros(shape=(au_length, 3, 4)))
      rot_quat.append(np.zeros(shape=(au_length, 4)))
      anchor_t_shape.append(np.zeros(shape=(au_length, 68 * 3)))

  if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl.pickle'))):
      os.remove(os.path.join('examples', 'dump', 'random_val_fl.pickle'))
  if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))):
      os.remove(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))
  if(os.path.exists(os.path.join('examples', 'dump', 'random_val_au.pickle'))):
      os.remove(os.path.join('examples', 'dump', 'random_val_au.pickle'))
  if (os.path.exists(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))):
      os.remove(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))

  with open(os.path.join('examples', 'dump', 'random_val_fl.pickle'), 'wb') as fp:
      pickle.dump(fl_data, fp)
  with open(os.path.join('examples', 'dump', 'random_val_au.pickle'), 'wb') as fp:
      pickle.dump(au_data, fp)
  with open(os.path.join('examples', 'dump', 'random_val_gaze.pickle'), 'wb') as fp:
      gaze = {'rot_trans':rot_tran, 'rot_quat':rot_quat, 'anchor_t_shape':anchor_t_shape}
      pickle.dump(gaze, fp)
  return au_emb, ains

def audio_2_landmarks(au_emb, shape_3d):
  model = Audio2landmark_model(opt_parser, jpg_shape=shape_3d)
  if(len(opt_parser.reuse_train_emb_list) == 0):
      model.test(au_emb=au_emb)
  else:
      model.test(au_emb=None)

def gen_video(scale, shift, img, img_path):
  fls = glob.glob1('examples', 'pred_fls_*.txt')
  fls.sort()

  for i in range(0,len(fls)):
      fl = np.loadtxt(os.path.join('examples', fls[i])).reshape((-1, 68,3))
      fl[:, :, 0:2] = -fl[:, :, 0:2]
      fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift

      if (ADD_NAIVE_EYE):
          fl = util.add_naive_eye(fl)

      # additional smooth
      fl = fl.reshape((-1, 204))
      fl[:, :48 * 3] = savgol_filter(fl[:, :48 * 3], 15, 3, axis=0)
      fl[:, 48*3:] = savgol_filter(fl[:, 48*3:], 5, 3, axis=0)
      fl = fl.reshape((-1, 68, 3))

      ''' STEP 6: Imag2image translation '''
      model = Image_translation_block(opt_parser, single_test=True)
      with torch.no_grad():
          model.single_test(jpg=img, fls=fl, filename=fls[i], prefix=img_path)
          print('finish image2image gen')
      os.remove(os.path.join('examples', fls[i]))

def get_video_path(ains, img_path):
  for ain in ains:
    OUTPUT_MP4_NAME = '{}_pred_fls_{}_audio_embed.mp4'.format(
      img_path,
      ain.split('.')[0]
      )
    mp4 = open('examples/{}'.format(OUTPUT_MP4_NAME),'rb').read()
  return 'examples/{}'.format(OUTPUT_MP4_NAME)

def generate_video_path(image_path, name):
  img = cv2.resize(cv2.imread(image_path), (256, 256))
  shape_3d, scale, shift, img = detect_landmarks(img)
  au_emb, ains = gen_audio()
  audio_2_landmarks(au_emb, shape_3d)
  gen_video(scale, shift, img, name)
  path = get_video_path(ains, name)
  return path

In [6]:
from fastapi import FastAPI,File, UploadFile, Form
from fastapi.responses import FileResponse
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import uuid
import os

app = FastAPI()

@app.get("/")
async def index():
    return {"Status": "Running"}

@app.post("/generate_video")
async def generate_video(
    name: str = Form(default=""),
    image: UploadFile = File(...),
    audio: UploadFile = File(...)
    ):
    print("Start")
    try:
        file_name1 = "examples/" + name + ".jpg"
        with open(file_name1, "wb") as f:
            content = image.file.read()
            f.write(content)
    except Exception as e:
        return {"response": False, "msg": str(e)}
    print("Done")
    try:
        file_name2 = "examples/M6_04_16k.wav"
        with open(file_name2, "wb") as f:
            content = audio.file.read()
            f.write(content)
    except Exception as e:
        return {"response": False, "msg": str(e)}

    file_name = generate_video_path(file_name1, name)
    return FileResponse(path=file_name, media_type='application/octet-stream', filename=file_name)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print("Please Copy above public url and past into config.py file of UI app.")
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [79]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://5fbe-35-197-105-77.ngrok.io


Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth


Start
Done


  0%|          | 0.00/85.7M [00:00<?, ?B/s]

...

Start
Done
Loaded the voice encoder model on cuda in 0.01 seconds.
Processing audio file M6_04_16k.wav
0 out of 0 are in this portion
Loaded the voice encoder model on cuda in 0.01 seconds.
source shape: torch.Size([1, 288, 80]) torch.Size([1, 256]) torch.Size([1, 256]) torch.Size([1, 288, 257])
converted shape: torch.Size([1, 288, 80]) torch.Size([1, 576])
Run on device: cuda
Loading Data random_val
EVAL num videos: 1
G: Running on cuda, total num params = 3.00M
======== LOAD PRETRAINED FACE ID MODEL examples/ckpt/ckpt_speaker_branch.pth =========
======== LOAD PRETRAINED FACE ID MODEL examples/ckpt/ckpt_content_branch.pth =========
48uYS3bHIA8
YAZuSHvwVC0
0yaLdVk_UyQ
E_kmpT-EfOg
fQR31F7L3ww
JPMZAOGGHh8
W6uRNCJmdtI
2KL8PfQPmBg
p575B7k07a8
iUoAe2gXKE4
HH-iOC056aQ
S8fiWqrZEew
ROWN2ssXek8
irx71tYyI-Q
me6cdZCM2FY
OkqHtWOFliM
OfPKHc6w2vw
1lh57VnuaKE
_ldiVrXgZKc
H1Xnb_rtgqY
45hn7-LXDX8
bs7ZWVqAGCU
UElg0R7fmlk
bCs5SoifsiY
1Lx_ZqrK1bM
RrnL6Pcjjbw
sRbWv2R2hxE
wJmdE0G4sEg
hE-4e1vEiT8
XXbxe3fCQq

Start
Done
Loaded the voice encoder model on cuda in 0.01 seconds.
Processing audio file M6_04_16k.wav
0 out of 0 are in this portion
Loaded the voice encoder model on cuda in 0.02 seconds.
source shape: torch.Size([1, 512, 80]) torch.Size([1, 256]) torch.Size([1, 256]) torch.Size([1, 512, 257])
converted shape: torch.Size([1, 512, 80]) torch.Size([1, 1024])
Run on device: cuda
Loading Data random_val
EVAL num videos: 1
G: Running on cuda, total num params = 3.00M
======== LOAD PRETRAINED FACE ID MODEL examples/ckpt/ckpt_speaker_branch.pth =========
======== LOAD PRETRAINED FACE ID MODEL examples/ckpt/ckpt_content_branch.pth =========
48uYS3bHIA8
YAZuSHvwVC0
0yaLdVk_UyQ
E_kmpT-EfOg
fQR31F7L3ww
JPMZAOGGHh8
W6uRNCJmdtI
2KL8PfQPmBg
p575B7k07a8
iUoAe2gXKE4
HH-iOC056aQ
S8fiWqrZEew
ROWN2ssXek8
irx71tYyI-Q
me6cdZCM2FY
OkqHtWOFliM
OfPKHc6w2vw
1lh57VnuaKE
_ldiVrXgZKc
H1Xnb_rtgqY
45hn7-LXDX8
bs7ZWVqAGCU
UElg0R7fmlk
bCs5SoifsiY
1Lx_ZqrK1bM
RrnL6Pcjjbw
sRbWv2R2hxE
wJmdE0G4sEg
hE-4e1vEiT8
XXbxe3fCQ

Start
Done
Loaded the voice encoder model on cuda in 0.01 seconds.
Processing audio file M6_04_16k.wav
0 out of 0 are in this portion
Loaded the voice encoder model on cuda in 0.01 seconds.
source shape: torch.Size([1, 512, 80]) torch.Size([1, 256]) torch.Size([1, 256]) torch.Size([1, 512, 257])
converted shape: torch.Size([1, 512, 80]) torch.Size([1, 1024])
Run on device: cuda
Loading Data random_val
EVAL num videos: 1
G: Running on cuda, total num params = 3.00M
======== LOAD PRETRAINED FACE ID MODEL examples/ckpt/ckpt_speaker_branch.pth =========
======== LOAD PRETRAINED FACE ID MODEL examples/ckpt/ckpt_content_branch.pth =========
48uYS3bHIA8
YAZuSHvwVC0
0yaLdVk_UyQ
E_kmpT-EfOg
fQR31F7L3ww
JPMZAOGGHh8
W6uRNCJmdtI
2KL8PfQPmBg
p575B7k07a8
iUoAe2gXKE4
HH-iOC056aQ
S8fiWqrZEew
ROWN2ssXek8
irx71tYyI-Q
me6cdZCM2FY
OkqHtWOFliM
OfPKHc6w2vw
1lh57VnuaKE
_ldiVrXgZKc
H1Xnb_rtgqY
45hn7-LXDX8
bs7ZWVqAGCU
UElg0R7fmlk
bCs5SoifsiY
1Lx_ZqrK1bM
RrnL6Pcjjbw
sRbWv2R2hxE
wJmdE0G4sEg
hE-4e1vEiT8
XXbxe3fCQ

Start
Done
Loaded the voice encoder model on cuda in 0.01 seconds.
Processing audio file M6_04_16k.wav
0 out of 0 are in this portion
Loaded the voice encoder model on cuda in 0.01 seconds.
source shape: torch.Size([1, 288, 80]) torch.Size([1, 256]) torch.Size([1, 256]) torch.Size([1, 288, 257])
converted shape: torch.Size([1, 288, 80]) torch.Size([1, 576])
Run on device: cuda
Loading Data random_val
EVAL num videos: 1
G: Running on cuda, total num params = 3.00M
======== LOAD PRETRAINED FACE ID MODEL examples/ckpt/ckpt_speaker_branch.pth =========
======== LOAD PRETRAINED FACE ID MODEL examples/ckpt/ckpt_content_branch.pth =========
48uYS3bHIA8
YAZuSHvwVC0
0yaLdVk_UyQ
E_kmpT-EfOg
fQR31F7L3ww
JPMZAOGGHh8
W6uRNCJmdtI
2KL8PfQPmBg
p575B7k07a8
iUoAe2gXKE4
HH-iOC056aQ
S8fiWqrZEew
ROWN2ssXek8
irx71tYyI-Q
me6cdZCM2FY
OkqHtWOFliM
OfPKHc6w2vw
1lh57VnuaKE
_ldiVrXgZKc
H1Xnb_rtgqY
45hn7-LXDX8
bs7ZWVqAGCU
UElg0R7fmlk
bCs5SoifsiY
1Lx_ZqrK1bM
RrnL6Pcjjbw
sRbWv2R2hxE
wJmdE0G4sEg
hE-4e1vEiT8
XXbxe3fCQq

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [79]


In [7]:
!ngrok authtoken 25duGEzkosaCsc5LAKsIbyvsh0j_25sWfgoc2ZefsE7kBJ4QC

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
